In [ ]:
# import relevant packages
import os
import cv2
import glob
import random
import scipy.io
import numpy as np
import pandas as pd
import sklearn.metrics
from itertools import product
import matplotlib.pyplot as plt
from tf_unet import image_util, unet, util



In [ ]:
# generate binary annotation masks of each of the four channels 
filenames = sorted(glob.glob('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataRaw/annotation_matfiles/*.mat'))

for f in filenames:
    mat_file = scipy.io.loadmat(f)
    labs = mat_file['Labels'].reshape(1,-1,1)
    pos = np.dstack(mat_file['Positions'])

    # data frame with label and position info
    mat = np.concatenate((labs, pos), axis = -1)
    mat_labs = np.uint16(mat[:,:,0]).reshape(-1,1)
    mat_x = np.uint16(mat[:,:,1]).reshape(-1,1)
    mat_x = mat_x - 1 # to match index of array
    mat_y = np.uint16(mat[:,:,2]).reshape(-1,1)
    mat_y = mat_y - 1 # to match index of array
    mat = pd.DataFrame(data=np.column_stack((mat_labs, mat_x, mat_y)), 
                       columns=['Labels','X','Y'])

    # separate based on label
    mat_1 = mat.loc[mat['Labels'] == 1]
    mat_2 = mat.loc[mat['Labels'] == 2]
    mat_3 = mat.loc[mat['Labels'] == 3]
    mat_4 = mat.loc[mat['Labels'] == 4]
    
    if f[90:96] == 'img091':
        mat_2.set_value(10, 'Y', 0)
        mat_2.set_value(98, 'Y', 0)
    
    zero_1 = np.zeros([500,500])
    zero_2 = np.zeros([500,500])
    zero_3 = np.zeros([500,500])
    zero_4 = np.zeros([500,500])

    # generate annotations
    zero_1[mat_1['Y'], mat_1['X']] = 255
    zero_2[mat_2['Y'], mat_2['X']] = 255
    zero_3[mat_3['Y'], mat_3['X']] = 255
    zero_4[mat_4['Y'], mat_4['X']] = 255
    
    base_name, ext = os.path.splitext(f)
    
    np.save(base_name[0:62] + 'DataProcessed/numpy_mask/' + 
            base_name[90:96] + '_1', zero_1)
    np.save(base_name[0:62] + 'DataProcessed/numpy_mask/' + 
            base_name[90:96] + '_2', zero_2)
    np.save(base_name[0:62] + 'DataProcessed/numpy_mask/' + 
            base_name[90:96] + '_3', zero_3)
    np.save(base_name[0:62] + 'DataProcessed/numpy_mask/' + 
            base_name[90:96] + '_4', zero_4)
    
    cv2.imwrite(base_name[0:62] + 'DataProcessed/binary_mask_final/' + 
                base_name[90:96] + '_1.png', zero_1)
    cv2.imwrite(base_name[0:62] + 'DataProcessed/binary_mask_final/' + 
                base_name[90:96] + '_2.png', zero_2)
    cv2.imwrite(base_name[0:62] + 'DataProcessed/binary_mask_final/' + 
                base_name[90:96] + '_3.png', zero_3)
    cv2.imwrite(base_name[0:62] + 'DataProcessed/binary_mask_final/' + 
                base_name[90:96] + '_4.png', zero_4)

# for image091, there are 2 Y coordinates greater than 500 
# (equals 65535, should be 0, indexes = 10 and 98)
# commands I need to use to fix after creating data frame (img091 only):
#    mat.set_value(10, 'Y', 0)
#    mat.set_value(98, 'Y', 0)
    
    

In [ ]:
# dilate the annotation masks
filenames = sorted(glob.glob('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/binary_mask_final/*.png'))

for f in filenames:
    img = cv2.imread(f, 0)
    kernel = np.ones((7,7), np.uint8)
    dilate = cv2.dilate(img, kernel, iterations = 1)
    base_name, ext = os.path.splitext(f)
    np.save(base_name[0:62] + 'DataProcessed/dilate_numpy_mask/' + 
            base_name[94:] + '_mask', dilate)
    cv2.imwrite(base_name[0:62] + 'DataProcessed/dilate_mask_final/' + 
                base_name[94:] + '_mask.png', dilate)
    
    

In [ ]:
# generate multi-class annotation masks
filenames = sorted(glob.glob('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/dilate_mask_final/*.png'))

for f in filenames:
    mask_1 = cv2.imread(f[0:101] + '1_mask.png',0)
    mask_2 = cv2.imread(f[0:101] + '2_mask.png',0)
    mask_3 = cv2.imread(f[0:101] + '3_mask.png',0)
    mask_4 = cv2.imread(f[0:101] + '4_mask.png',0)
    
    mask_merge = np.maximum.reduce([mask_1, mask_2, mask_3, mask_4])
    mask_merge_final = ~mask_merge
    mask_stack = np.dstack((mask_1, mask_2, mask_3, mask_4, mask_merge_final))
    
    np.save(f[0:76] + 'dilate_numpy_merge/' + 
            f[94:101] + 'mask', mask_stack)
    cv2.imwrite(f[0:76] + 'dilate_merge_final/' + 
                f[94:101] + 'mask.png', mask_merge)
    
    

In [ ]:
# generate black and white images from RGB images
filenames = sorted(glob.glob('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataRaw/image_data/*.png'))

for f in filenames:
    img = cv2.imread(f)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(f[0:70] + 'image_data_gray/' + f[81:], img_gray)



In [ ]:
# generate randomly cropped and rotated images
def randomCrop(img, mask, width, height):
    assert img.shape[0] >= height
    assert img.shape[1] >= width
    assert img.shape[0] == mask.shape[0]
    assert img.shape[1] == mask.shape[1]
    x = random.randint(0, img.shape[1] - width)
    y = random.randint(0, img.shape[0] - height)
    img = img[y:y+height, x:x+width]
    mask = mask[y:y+height, x:x+width]
    return img, mask

# read in files and masks
filenames = sorted(glob.glob('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataRaw/image_data/*.png'))
masknames = sorted(glob.glob('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/dilate_merge_final/*.png'))

np.random.seed(1212)
for f in filenames:
        for m in masknames:
            img = cv2.imread(f)
            mask = cv2.imread(m,0)

            if (m[95:101] == f[81:87]):
                crop_img1, crop_mask1 = randomCrop(img, mask, 350, 350)

                # prepare rotation matrices
                rows, cols = crop_mask1.shape
                M90 = cv2.getRotationMatrix2D((cols/2,rows/2),90,1)
                M180 = cv2.getRotationMatrix2D((cols/2,rows/2),180,1)
                M270 = cv2.getRotationMatrix2D((cols/2,rows/2),270,1)
                
                # rotate cropped images and masks
                # 90 degree rotation
                crop_img2, crop_mask2 = randomCrop(img, mask, 350, 350)
                crop_rot_img2 = cv2.warpAffine(crop_img2, M90, (cols,rows))
                crop_rot_mask2 = cv2.warpAffine(crop_mask2, M90, (cols,rows))

                #180 degree rotation
                crop_img3, crop_mask3 = randomCrop(img, mask, 350, 350)
                crop_rot_img3 = cv2.warpAffine(crop_img3, M180, (cols,rows))
                crop_rot_mask3 = cv2.warpAffine(crop_mask3, M180, (cols,rows))

                # 270 degree rotation
                crop_img4, crop_mask4 = randomCrop(img, mask, 350, 350)
                crop_rot_img4 = cv2.warpAffine(crop_img4, M270, (cols,rows))
                crop_rot_mask4 = cv2.warpAffine(crop_mask4, M270, (cols,rows))

                
                cv2.imwrite('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/crop_images/crop' + f[84:87] + '1.png', crop_img1)
                cv2.imwrite('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/crop_masks/crop' + f[84:87] + '1_mask.png', crop_mask1)
                cv2.imwrite('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/crop_images/crop' + f[84:87] + '2.png', crop_rot_img2)
                cv2.imwrite('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/crop_masks/crop' + f[84:87] + '2_mask.png', crop_rot_mask2)
                cv2.imwrite('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/crop_images/crop' + f[84:87] + '3.png', crop_rot_img3)
                cv2.imwrite('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/crop_masks/crop' + f[84:87] + '3_mask.png', crop_rot_mask3)
                cv2.imwrite('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/crop_images/crop' + f[84:87] + '4.png', crop_rot_img4)
                cv2.imwrite('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/crop_masks/crop' + f[84:87] + '4_mask.png', crop_rot_mask4)



In [ ]:
# determine weights for weighted cross entropy
label_train = np.array([cv2.imread(file,0) for file in glob.glob('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/train/*_mask.png')])

# sum of all labeled pixels
np.sum(label_train)/255



In [ ]:
# percentage of all labeled cells
(np.sum(label_train)/255)/(60*350*350) # about 6%

# weight given to foreground class
1/((np.sum(label_train)/255)/(60*350*350))



In [ ]:
# train TensorFlow Unet (Model 1)
# verification batch size = 4, dropout = 0.75
# set seed
np.random.seed(1212)

# set up data and annotation masks
# channels = 3, n_class = 2
data_provider = image_util.ImageDataProvider(search_path = '/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/train/*.png',
                                             data_suffix = '.png', 
                                             mask_suffix = '_mask.png')
net = unet.Unet(channels=data_provider.channels, n_class=data_provider.n_class, 
                layers=3, features_root=32, cost = 'cross_entropy', 
                cost_kwargs = {'class_weight': [0.0625, 0.9375]})
trainer = unet.Trainer(net, optimizer = "adam",
                       opt_kwargs = dict(learning_rate = 0.0001))
path = trainer.train(data_provider, "/Users/piper/unet_train", 
                     training_iters=20, epochs=100) 


In [ ]:
# train TensorFlow Unet (Model 2)
# verification batch size = 4, dropout = 0.9

# set seed
np.random.seed(1212)

# set up data and annotation masks
# channels = 3, n_class = 2
data_provider = image_util.ImageDataProvider(search_path = '/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/train/*.png',
                                             data_suffix = '.png', 
                                             mask_suffix = '_mask.png')
net = unet.Unet(channels=data_provider.channels, n_class=data_provider.n_class, 
                layers=3, features_root=32, cost = 'cross_entropy', 
                cost_kwargs = {'class_weight': [0.0625, 0.9375]})
trainer = unet.Trainer(net, optimizer = "adam", 
                       opt_kwargs = dict(learning_rate = 0.0001))
path = trainer.train(data_provider, "/Users/piper/unet_train", dropout = 0.9, 
                     training_iters=20, epochs=100) 



In [ ]:
# train TensorFlow Unet (Model 3)
# verification batch size = 8, dropout = 0.75
# set seed
np.random.seed(1212)

# set up data and annotation masks
# channels = 3, n_class = 2
data_provider = image_util.ImageDataProvider(search_path = '/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/train/*.png',
                                             data_suffix = '.png', 
                                             mask_suffix = '_mask.png')
net = unet.Unet(channels=data_provider.channels, n_class=data_provider.n_class, 
                layers=3, features_root=32, cost = 'cross_entropy', 
                cost_kwargs = {'class_weight': [0.0625, 0.9375]})
trainer = unet.Trainer(net, optimizer = "adam", verification_batch_size = 8, 
                       opt_kwargs = dict(learning_rate = 0.0001))
path = trainer.train(data_provider, "/Users/piper/unet_train", 
                     training_iters=20, epochs=100) 



In [ ]:
# train TensorFlow Unet (Model 4)
# verification batch size = 8, dropout = 0.9
# set seed
np.random.seed(1212)

# set up data and annotation masks
# channels = 3, n_class = 2
data_provider = image_util.ImageDataProvider(search_path = '/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/train/*.png',
                                             data_suffix = '.png', 
                                             mask_suffix = '_mask.png')
net = unet.Unet(channels=data_provider.channels, n_class=data_provider.n_class, 
                layers=3, features_root=32, cost = 'cross_entropy', 
                cost_kwargs = {'class_weight': [0.0625, 0.9375]})
trainer = unet.Trainer(net, optimizer = "adam", verification_batch_size = 8, 
                       opt_kwargs = dict(learning_rate = 0.0001))
path = trainer.train(data_provider, "/Users/piper/unet_train", dropout = 0.9, 
                     training_iters=20, epochs=100) 


In [ ]:
# apply model to test data and predict classes
# results shown from model 1 only
# set seed
np.random.seed(1212)
data_test_provider = image_util.ImageDataProvider(search_path = '/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataProcessed/test/*.png', data_suffix = '.png', mask_suffix = '_mask.png')
data_test, label_test = data_test_provider(15)

# predict classes
pred = net.predict('/Users/piper/Unet_Models/Model3/unet_train', data_test)



In [ ]:
# shape of predictions
np.shape(pred)



In [ ]:
# maximum predicted value for foreground class
np.max(pred[:,:,:,1])



In [ ]:
# crop test annotation masks to match predictions
pred_final = pred[:,:,:,1]
label_test_final = util.crop_to_shape(label_test, pred.shape)
label_test_final = label_test_final[:,:,:,1]



In [ ]:
# check that predictions and labels are same shape
np.shape(pred_final) == np.shape(label_test_final)



In [ ]:
# binarize the predictions
for i in range(0,15):
    for j in range(0,460):
        for k in range(0, 460):
            if pred_final[i, j, k] > 0.5:
                pred_final[i, j, k] = 1
            else:
                pred_final[i, j, k] = 0

                

In [ ]:
# calculate relevant metrics
# reshape 3D arrays into 1D array
pred_final_1d = pred_final.reshape(-1)
label_test_final_1d = label_test_final.reshape(-1)

# accuracy
acc = sklearn.metrics.accuracy_score(label_test_final_1d, pred_final_1d)
acc



In [ ]:
# precision
prec = sklearn.metrics.precision_score(label_test_final_1d, pred_final_1d)
prec


In [ ]:
# recall
recall = sklearn.metrics.recall_score(label_test_final_1d, pred_final_1d)
recall



In [ ]:
# F1 score
f1_score = sklearn.metrics.f1_score(label_test_final_1d, pred_final_1d)
f1_score



In [ ]:
# plot some predictions
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize = (12,5))
ax[0].imshow(data_test[0,...,0], aspect="auto", cmap = 'gray')
ax[1].imshow(label_test[0,...,1], aspect="auto", cmap = 'gray')
mask = pred[0,...,1] > 0.5
ax[2].imshow(mask, aspect="auto", cmap = 'gray')
ax[0].set_title("Input")
ax[1].set_title("Ground truth")
ax[2].set_title("Prediction")
fig.tight_layout()



In [ ]:
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize = (12,5))
ax[0].imshow(data_test[2,...,0], aspect="auto", cmap = 'gray')
ax[1].imshow(label_test[2,...,1], aspect="auto", cmap = 'gray')
mask = pred[2,...,1] > 0.5
ax[2].imshow(mask, aspect="auto", cmap = 'gray')
ax[0].set_title("Input")
ax[1].set_title("Ground truth")
ax[2].set_title("Prediction")
fig.tight_layout()



In [ ]:
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize = (12,5))
ax[0].imshow(data_test[5,...,0], aspect="auto", cmap = 'gray')
ax[1].imshow(label_test[5,...,1], aspect="auto", cmap = 'gray')
mask = pred[5,...,1] > 0.5
ax[2].imshow(mask, aspect="auto", cmap = 'gray')
ax[0].set_title("Input")
ax[1].set_title("Ground truth")
ax[2].set_title("Prediction")
fig.tight_layout()



In [ ]:
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize = (12,5))
ax[0].imshow(data_test[8,...,0], aspect="auto", cmap = 'gray')
ax[1].imshow(label_test[8,...,1], aspect="auto", cmap = 'gray')
mask = pred[8,...,1] > 0.5
ax[2].imshow(mask, aspect="auto", cmap = 'gray')
ax[0].set_title("Input")
ax[1].set_title("Ground truth")
ax[2].set_title("Prediction")
fig.tight_layout()



In [ ]:
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize = (12,5))
ax[0].imshow(data_test[11,...,0], aspect="auto", cmap = 'gray')
ax[1].imshow(label_test[11,...,1], aspect="auto", cmap = 'gray')
mask = pred[11,...,1] > 0.5
ax[2].imshow(mask, aspect="auto", cmap = 'gray')
ax[0].set_title("Input")
ax[1].set_title("Ground truth")
ax[2].set_title("Prediction")
fig.tight_layout()



In [ ]:
# generate image as example of image and labels
# read in one image for example (img_068)
img = cv2.imread('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataRaw/image_data/img068.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# read in corresponding annotation file
mat = scipy.io.loadmat('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataRaw/annotation_matfiles/img068_annotations.mat')
labs = mat['Labels'].reshape(1,-1,1)
pos = np.dstack(mat['Positions'])

# create variables of interest
mat = np.concatenate((labs, pos), axis = -1)
mat_labs = np.uint16(mat[:,:,0]).reshape(-1,1)
mat_x = np.uint16(mat[:,:,1]).reshape(-1,1)
mat_y = np.uint16(mat[:,:,2]).reshape(-1,1)

# create Pandas data frame
mat = pd.DataFrame(data=np.column_stack((mat_labs, mat_x, mat_y)), 
                   columns=['Labels','X','Y'])

# show image with labeled cells
classes = ['','Y','Z','X']
unique = np.unique(classes)
colors = {1:'blue', 2:'forestgreen', 3:'darkorange', 4:'red'}
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(img)
ax.scatter(mat['X'], mat['Y'], s = 20, 
           c = mat['Labels'].apply(lambda x: colors[x]))
ax.axis('off')
plt.show()



In [ ]:
# make graph to understand distribution of classes
filenames = sorted(glob.glob('/Users/piper/Piper Documents/Biomedical Imaging/Final Project/DataRaw/annotation_matfiles/*.mat'))

labs_all = []

for f in filenames:
    mat_file = scipy.io.loadmat(f)
    labs = mat_file['Labels']
    labs_all.append(labs)
    
# Concatenate all data into one DataFrame
combine_labs = np.concatenate((labs_all[0:29]), axis=None)
labs_data = pd.DataFrame(combine_labs, columns = ['Label'])

# change labels 
change_labels = {'Label': {1:'Epithelial', 2:'Fibroblast', 3:'Inflammatory', 4:'Other'}}
labs_data.replace(change_labels, inplace=True)

# create histogram showing distribution of cell types
pd.value_counts(labs_data['Label'])



In [ ]:
# total number of cells
np.shape(labs_data)[0]



In [ ]:
summary = {'Label':['Epithelial','Fibroblast','Inflammatory','Other'], 'Frequency':[4043,3280,3251,430]} 
summary_data = pd.DataFrame(summary)
plt.figure(figsize=(8,8))
plt.bar(summary_data['Label'], summary_data['Frequency'])
plt.xlabel('Nuclei Label')
plt.ylabel('Frequency')
plt.show()

